# **Car Price Prediction**

**Mục tiêu:**
Xây dựng Pipeline học máy truyền thống cho vấn đề trên.

1.  **EDA (Exploratory Data Analysis):**
    *   Thống kê mô tả dữ liệu, trực quan hóa để hiểu rõ các biến.
    *   Phát hiện dữ liệu thiếu, giá trị ngoại lai và xu hướng phân phối.

2.  **Tiền xử lý dữ liệu (Data Preprocessing):**
    *   Làm sạch dữ liệu, xử lý các giá trị bị thiếu.
    *   Mã hóa dữ liệu phân loại (One-Hot Encoding, Label Encoding).
    *   Chuẩn hóa dữ liệu số (StandardScaler, MinMaxScaler).
    *   Chia tập dữ liệu thành tập huấn luyện (train) và tập kiểm tra (test).

3.  **Trích xuất và lựa chọn đặc trưng (Feature Engineering & Selection):**
    *   Tạo ra các đặc trưng mới có ý nghĩa hơn từ dữ liệu gốc (ví dụ: tuổi của xe).
    *   Lựa chọn những đặc trưng quan trọng nhất để đưa vào mô hình.

4.  **Huấn luyện mô hình (Model Training):**
    *   Áp dụng các thuật toán học máy phù hợp cho bài toán **hồi quy (regression)** như:
        *   Logistic Regression
        *   SVM
        *   Random Forest

5.  **Đánh giá mô hình (Model Evaluation):**
    *   Sử dụng các chỉ số đánh giá cho bài toán hồi quy:
        *   Accuracy
        *   Precision
        *   Recall
        *   F1-score

**Yêu cầu:**
* Các nhóm không được chọn trùng tập dữ liệu.
* Tập dữ liệu phải có missing value để sinh viên thực hành kỹ thuật imputation.
* Tập dữ liệu phải có categorical value để sinh viên thực hành các kỹ thuật
encoding.
* Số lượng mẫu (sample size) đủ lớn để pipeline có ý nghĩa; việc lựa chọn cụ thể
nên được thảo luận trực tiếp với giảng viên trên lớp.

**Nhiệm vụ:** mỗi nhóm phải xây dựng một pipeline học máy truyền thống cho
dữ liệu dạng bảng. Pipeline này phải được thiết kế sao cho cho phép cấu hình các kỹ
thuật và tham số ở từng bước. Ví dụ:
* Scaling: có thể lựa chọn MinMaxScaler hoặc StandardScaler; nếu chọn MinMaxScaler,
sinh viên cần cấu hình feature_range.
* Giảm số chiều: có thể lựa chọn PCA với các mức giữ lại phương sai khác nhau
(90%, 95%, ...).
* Mô hình: có thể lựa chọn Logistic Regression, SVM, Random Forest và so sánh
kết quả.

Kết quả cuối cùng cần bao gồm: báo cáo phân tích EDA, mô tả pipeline, các tham
số đã thử nghiệm, và so sánh hiệu quả giữa các cấu hình.

## **1. Thiết lập môi trường và Tải dữ liệu**



### **1.1 Import thư viện và cấu hình Kaggle API từ Secrets**

In [19]:
import os
import pandas as pd
from google.colab import userdata
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
print("Các thư viện đã được import và cấu hình.")

# Tự động cấu hình Kaggle API
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

Các thư viện đã được import và cấu hình.


### **1.2 Tải, giải nén và đọc dữ liệu**

In [26]:
!pip install kaggle --quiet
!kaggle datasets download -d deepcontractor/car-price-prediction-challenge -p ./data

# Thêm cờ -o để tự động ghi đè file cũ, tránh bị hỏi và gây lỗi
!unzip -o -q ./data/car-price-prediction-challenge.zip -d ./data

# Đọc file 'train.csv' (đây là file dữ liệu huấn luyện)
try:
    df = pd.read_csv('./data/car_price_prediction.csv')
    print("\nĐọc dataset thành công!")
except FileNotFoundError:
    print("\nLỗi: Không tìm thấy dataset. Vui lòng kiểm tra lại kết quả lệnh 'ls' ở trên.")

Dataset URL: https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge
License(s): CC0-1.0
car-price-prediction-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)

Đọc dataset thành công!


## **2. EDA (Exploratory Data Analysis)**

### **2.1 Xem thông tin tổng quan (kiểu dữ liệu, số lượng giá trị không rỗng)**

In [32]:
print("\nThông tin tổng quan về các cột:")
df.info()


Thông tin tổng quan về các cột:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color  

### **2.2 Phân phối dữ liệu của từng cột**

Trong phần này, chúng ta sẽ trực quan hóa phân phối của từng cột để hiểu rõ hơn về đặc điểm của chúng.
- **Biến số:** Sử dụng biểu đồ Histogram (để xem hình dạng phân phối) và Boxplot (để phát hiện giá trị ngoại lai).
- **Biến phân loại:** Sử dụng biểu đồ Countplot (để xem tần suất của từng danh mục).